For example, consider the following records, which have already been organized into chronological order:

```
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up

```

Timestamps are written using year-month-day hour:minute format. The guard falling asleep or waking up is always the one whose shift most recently started. Because all asleep/awake times are during the midnight hour (00:00 - 00:59), only the minute portion (00 - 59) is relevant for those events.

Visually, these records show that the guards are asleep at these times:

```
Date   ID   Minute
            000000000011111111112222222222333333333344444444445555555555
            012345678901234567890123456789012345678901234567890123456789
11-01  #10  .....####################.....#########################.....
11-02  #99  ........................................##########..........
11-03  #10  ........................#####...............................
11-04  #99  ....................................##########..............
11-05  #99  .............................................##########.....
```

The columns are Date, which shows the month-day portion of the relevant day; ID, which shows the guard on duty that day; and Minute, which shows the minutes during which the guard was asleep within the midnight hour. (The Minute column's header shows the minute's ten's digit in the first row and the one's digit in the second row.) Awake is shown as ., and asleep is shown as #.

Note that guards count as asleep on the minute they fall asleep, and they count as awake on the minute they wake up. For example, because Guard #10 wakes up at 00:25 on 1518-11-01, minute 25 is marked as awake.

If you can figure out the guard most likely to be asleep at a specific time, you might be able to trick that guard into working tonight so you can have the best chance of sneaking in. You have two strategies for choosing the best guard/minute combination.

In [1]:
def read_input_file(filename):
    with open(filename) as f:
        for line in f:
            yield line

In [5]:
lines = []
for line in read_input_file("input_4.txt"):
    lines.append(line)

In [18]:
def seperate_dates_n_strings(line):
    string = line.split("] ")
    date = string[0]
    work = string[1].strip("\n")
    date = date.strip("[")
    # removing year
    date = date.split("-", 1)[1] # splitting with first occurence of "-"
    return [date, work]
    
seperate_dates_n_strings("[1518-07-04 00:01] falls asleep")

['07-04 00:01', 'falls asleep']

In [167]:
dates_n_works = []
for line in lines:
    dates_n_works.append(seperate_dates_n_strings(line))

dates_n_works = sorted(dates_n_works, key=lambda x: x[0])
# print(dates_n_works)
seperation_string = ""
for item in dates_n_works:
    seperation_string += item[0] + " " + item[1] + "\n"

strings = seperation_string.split("Guard")
del strings[0]
strings = [s.strip() for s in strings]

ted_strings = []
last_string = strings[len(strings) - 1]
last_string = last_string.split("\n")
for s in strings:
    s = s.split("\n")
    del s[len(s) - 1]
    ted_strings.append(s)

ted_strings.pop()
ted_strings.append(last_string)
ted_strings[len(ted_strings) - 1]

# print(ted_strings)

['#613 begins shift', '11-23 00:47 falls asleep', '11-23 00:51 wakes up']

In [74]:
import re

def get_guard_id(line):
    return re.findall("#[0-9]*", line)[0].strip("#")

def assign_timings_to_guard_id(list_of_lines):
    guard_id = get_guard_id(list_of_lines[0])
    return {guard_id: list_of_lines[1:]}
    
assign_timings_to_guard_id(['#773 begins shift', '03-31 00:17 falls asleep', '03-31 00:42 wakes up'])

{'773': ['03-31 00:17 falls asleep', '03-31 00:42 wakes up']}

In [114]:
list_of_sleeping_guard_timings = []
for string in ted_strings:
    list_of_sleeping_guard_timings.append(assign_timings_to_guard_id(string))

# for item in list_of_sleeping_guard_timings:
#     print(item)

In [104]:
def get_sleep_timings(list_of_strings):
    inner_list = []
    for strings in batch_gen(list_of_strings, 2):
        if "falls asleep" in strings[0]:
            day, minutes_s = get_minutes(strings[0])
#             print(minutes_s)
        if "wakes up" in strings[1]:
            day, minutes_e = get_minutes(strings[1])
#             print(minutes_e)
        inner_list.append((day, int(minutes_s), int(minutes_e)))
    return inner_list

def batch_gen(data, batch_size):
    for i in range(0, len(data), batch_size):
            yield data[i : i + batch_size]
            
def get_minutes(line):
    timings = line.split()
    day = timings[0]
    hour = timings[1]
    minutes = hour.split(":")[1]
    return day, minutes

get_sleep_timings(['04-15 00:10 falls asleep', '04-15 00:19 wakes up', '04-15 00:22 falls asleep', '04-15 00:56 wakes up'])

[('04-15', 10, 19), ('04-15', 22, 56)]

In [132]:
guard_sleep_timings = {}
for l in list_of_sleeping_guard_timings:
    for k, v in l.items():
        if len(v) % 2 == 0:
            if k not in guard_sleep_timings.keys():
                guard_sleep_timings[k] = get_sleep_timings(v)
            elif k in guard_sleep_timings.keys():
                guard_sleep_timings[k] += get_sleep_timings(v)
        else:
            print("------PROBLEM-----")
            
# print(guard_sleep_timings)

def get_sleep_duration(tuple_sleep):
    day, s, e = tuple_sleep
    if e > s:
        return e - s
# print(get_sleep_duration(('10-15', 17, 23)))

In [131]:
guards_sleep_dur = {}
for k, v in guard_sleep_timings.items():
    duration = 0
    if len(v) > 0:
        for item in v:
            duration += get_sleep_duration(item)
    if duration != 0:
        guards_sleep_dur[k] = duration

max_sleeping_minutes = 0
for k, v in guards_sleep_dur.items():
    if v > max_sleeping_minutes:
#         print(k)
        max_sleeping_minutes = v
# max_sleeping_minutes
guards_sleep_dur['1217']

482

from above analysis we came to know that `guard '1217'` has highest minutes of sleep

In [146]:
# print(guard_sleep_timings['1217'])
sleep_day = {}
for i in guard_sleep_timings['1217']:
    day, s, e = i
    inner_list = []
    for i in range(s, e):
        inner_list.append(i)
    if day not in sleep_day.keys():
        sleep_day[day] = inner_list
    elif day in sleep_day.keys():
        sleep_day[day] += inner_list

list_of_sets = []
for k, v in sleep_day.items():
    list_of_sets += v

# print(list_of_sets)
from collections import Counter

def get_most_common(list_of_sets):
    data = Counter(list_of_sets)
    return max(list_of_sets, key=data.get)

print(get_most_common(list_of_sets))

40


so `40` minute is the most_common minute by `1217`

In [147]:
print(1217 * 40)

48680


## Part two

Strategy 2: Of all guards, which guard is most frequently asleep on the same minute?

In the example above, Guard #99 spent minute 45 asleep more than any other guard or minute - three times in total. (In all other cases, any guard spent any minute asleep at most twice.)

In [165]:
guard_minutes = {}
for k, v in guard_sleep_timings.items():
    inner_list = []
    for item_tuple in v:
        day, s, e = item_tuple
        i_list = []
        for i in range(s, e):
            i_list.append(i)
        inner_list += i_list
    guard_minutes[k] = inner_list
#     print(k, inner_list)

dict_most_common = {}
for k, v in guard_minutes.items():
    if len(v) > 0:
        data = Counter(v)
        most_common = max(v, key=data.get)
        most_common_count = dict(data)[most_common]
        dict_most_common[k] = (most_common, most_common_count)
        
for k, v in dict_most_common.items():
    print(k, v)

773 (33, 9)
2963 (35, 12)
643 (43, 8)
1129 (20, 6)
101 (42, 11)
811 (44, 6)
449 (38, 7)
1049 (36, 10)
613 (45, 8)
3023 (12, 7)
197 (22, 6)
2297 (48, 10)
353 (51, 5)
2333 (53, 8)
1117 (50, 6)
1657 (10, 3)
1217 (40, 16)
2789 (34, 17)
3163 (18, 5)
277 (43, 10)


from above we can say that guard `2789` slept `17` times on `34` minute

In [166]:
print(2789 * 34)

94826
